# Using Self-Organizing Maps to solve the Traveling Salesman Problem

**Traveling Salesman Problem**
- NP-Complete   
- Traverses all cities in a given map only once .  
- Difficulty increases with increase in Number of cities   

![SOM](som.png)

**Self Organizing Maps**
- SOM is a grid of nodes. Closely related to the idea of a model, that is, the real-world observation the map is trying to represent.   
The purpose is to represent the model with a lower number of dimensions, while maintaining the relations of similarity of the nodes contained in it.   
   
- More similar the nodes, more spatially closer they are organized.Hence, it makes SOM good for pattern visualization and organization of data.   
   
- To obtain the structure, the map is applied a regression operation to modify the nodes position in order to update the nodes, one element from the model at a time.   
   
- The position of the node is updated adding the distance from it to the given element x Neighbourhood Factor of the winner Neuron. 

- SOMs are used for Dimensionality Reduction and Dense Vector Representations of the data.   
   
- They are different from the Neural Networks because of the technique they used to learn, unlike NN which uses Backpropogation with Gradient Descent, **SOMs use Neigbourhood-based techniques to preserve topological properties of the input space.**   
   
- SOMs retain Topology and reveal correlations. They Classify data without Supervision. No Target Vector -> No Backprop. No Lateral connections (No Neural Network connection) between the output nodes. 

### Algorithm:   
1. Each node's weights are initialized.   
2. A vector is chosen at random from the set of training data.   
3. Every node is examined to calculate which one's weight are most like the input vector. The winning node is commonly known as the **BEST Matching Unit (BMU)**
4. Then the neighbourhood of the BMU is calculated. The amount of neighbors decrease over the time.   
5. The winning Node is rewarded with becoming more like the sample vector. The neighbours also become more like the sample vector. The closer a node is to the BMU, the more its weight get altered and the farther away the neighbor is from the BMU the less it learns.   
6. Repeat step 2 for N Iterations.   
   
   
***Best Matching Unit*** is a technique which calculates the distance from each weight to the sample vector, by running through all weight vectors. The weight with the shortest distance is the winner.   

**Modifying the technique:** 
To use the network to solve the TSP, the main concept is to understand how to modify the neighbourhood function. If instead of a grid we declare a circular array of neurons, each node will only be concious of the neurons in front of and behind it. That is, the inner similarity will work just in one dimension. Making this slight modification, the SOM will behave as an elastic ring, getting closer to the cities but trying to minimize the perimeter of it thanks to the neighbourhood function.   

**NEIGHBORHOOD FUNCTION AND LEARNING RATE**   
- It is used to control the exploration and exploitation of the algorithm.   
- To obtain high exploration first and high exploitation after that in the execution, we must include a decay in both the neighborhood function and the learning rate.   
- Decaying the Learning rate will ensure less aggressive displacement of the neurons around the model.   
- Decaying the neighbourhood will result in a more moderate exploitation of the local minima of each part of the moddel. 

**TO ASSOCIATE THE CITY WITH ITS WINNER NEURON, TRANSVERSE THE RING STARTING FROM ANY POINT AND SORT THE CITIES BY ORDER OF APPEARANCE OF THEIR WINNER NEURON IN THE RING. IF SEVERAL CITIES MAP TO THE SAME NEURON, IT IS BECAUSE THE ORDER OF TRANSVERSING SUCH CITIES HAVE NOT BEEN COMTEMPLATED BY THE SOM. IN THAT CASE, ANY POSSIBLE ORDER CAN BE CONSIDERED.**

## Code

In [3]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
class SelfOrganizingMap: 
    def read_data(self, count):
        df = pd.read_csv('../data.csv', header=None) 
        nodes = []
        for i in range(len(df[0])):
            sp = df[0][i].split(' ')
            x = float(sp[1])
            y = float(sp[2])
            nodes.append([x, y])
        nodes = nodes[:count]   
        return pd.DataFrame(nodes, columns=['x', 'y'])
    
    
    def normalize(self, points):
        """Return the normalized version of a given vector of points"""
        ratio = (points['x'].max() - points['x'].min()) / (points['y'].max() - points['y'].min()), 1
        ratio = np.array(ratio) / max(ratio)
        norm = points.apply(lambda c: (c - c.min()) / (c.max() - c.min()))
        return norm.apply(lambda p: ratio * p, axis=1)
    
    
    def select_closest(self, candidates, origin):
        """Return the index of the closest candidate to a given point"""
        return self.eucledian_distance(candidates, origin).argmin()

    
    def eucledian_distance(self, a, b):
        return np.linalg.norm(a - b, axis = 1)
    
    
    def route_distance(self, cities):
        """Return the cost of traversing a route of cities in a certain order"""
        points = cities[['x', 'y']]
        distances = self.eucledian_distance(points, np.roll(points, 1, axis=0))
        return np.sum(distances)
    
    
    
    def generate_network(self, size):
        """
        Generate neuron network for a given size
        Return a vector of 2-D points in the interval of [0,1]
        """
        return np.random.rand(size, 2)

    
    def get_neighborhood(self, center, radix, domain):
        """Get the range gaussian of given radix around a center index"""
        if radix < 1: radix = 1
        deltas = np.absolute(center - np.arange(domain))
        distances = np.minimum(deltas, domain - deltas)
        return np.exp(-(distances*distances) / (2*(radix*radix)))

    
    def get_route(self, cities, network):
        """Return the route computed by a network"""
        cities['winner'] = cities[['x', 'y']].apply(
            lambda c: self.select_closest(network, c), 
            axis=1, raw=True)
        return cities.sort_values('winner').index
    
        
        
    def som(self, problem, iterations, learning_rate=0.8):
        """Solve the TSP using a Self-Organizing Map."""

        # Obtain the normalized set of cities (w/ coord in [0,1])
        cities = problem.copy()

        cities[['x', 'y']] = self.normalize(cities[['x', 'y']])

        # The population size is 8 times the number of cities
        n = cities.shape[0] * 8

        # Generate an adequate network of neurons:
        network = self.generate_network(n)
        #print('Network of {} neurons created. Starting the iterations:'.format(n))

        for i in range(iterations):
            if not i % 100:
                print('\t> Iteration {}/{}'.format(i, iterations), end="\r")
            # Choose a random city
            city = cities.sample(1)[['x', 'y']].values
            winner_idx = self.select_closest(network, city)
            # Generate a filter that applies changes to the winner's gaussian
            gaussian = self.get_neighborhood(winner_idx, n//10, network.shape[0])
            # Update the network's weights (closer to the city)
            network += gaussian[:,np.newaxis] * learning_rate * (city - network)
            # Decay the variables
            learning_rate = learning_rate * 0.99997
            n = n * 0.9997

            # Check if any parameter has completely decayed.
            if n < 1:
                #print('Radius has completely decayed, finishing execution',
                #'at {} iterations'.format(i))
                break
            if learning_rate < 0.001:
                #print('Learning rate has completely decayed, finishing execution',
                #'at {} iterations'.format(i))
                break
        else:
            pass
        route = self.get_route(cities, network)
        self.plot_route(cities, route, 'route.png')
        return route


In [10]:
start = time.time()

maps = SelfOrganizingMap()

nodes = maps.read_data(500)
route = maps.som(nodes, 100000)
nodes1 = nodes.reindex(route)
distance = maps.route_distance(nodes1)

print('Shortest distance', distance)
time_lapse = time.time() - start
print('Time spend for finding shortest distance', time_lapse)

Network of 4000 neurons created. Starting the iterations:
Radius has completely decayed, finishing execution at 27642 iterations
Shortest distance 19988.001850987184
Time spend for finding shortest distance 42.00268220901489


References: 
1. https://github.com/DiegoVicen/som-tsp   
2. https://towardsdatascience.com/self-organizing-maps-ff5853a118d4
3. https://diego.codes/post/som-tsp/   
4. https://www.youtube.com/watch?v=9ZhwKv_bUx8